### Quick Description

This notebook calculates the toxicity score from each sentence in a subtitles' file. Finally, it persists the data in a DataFrame like output.

In [1]:
import yaml
import glob
import os
import pandas as pd

from tqdm import tqdm
from textblob import TextBlob

In [2]:
dataset = "coraal"

filepaths = yaml.load(open("/home/guilherme/Desktop/dissertation/config/filepaths.yaml"))

/tmp/ipykernel_511789/670096696.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  filepaths = yaml.load(open("/home/guilherme/Desktop/dissertation/config/filepaths.yaml"))


In [3]:
def calculate_toxicity_scores(save_dir):
    filenames = [file.split("/")[-1] for file in glob.glob(os.path.join(save_dir, "*"))]

    for filename in tqdm(filenames, total=len(filenames)):
        df_sentences = pd.read_csv(os.path.join(save_dir, filename))
        for i, row in df_sentences.iterrows():
            df_sentences.loc[i, "textblob_score"] = TextBlob(row.text).sentiment.polarity

        df_sentences["textblob_score"] = 1 - df_sentences.textblob_score
        
        # Because 2 is the absolute maximum value
        df_sentences["textblob_score"] = (
            (df_sentences["textblob_score"] - df_sentences["textblob_score"].min())
            / (2 - df_sentences["textblob_score"].min())
        )

        df_sentences.to_csv(os.path.join(save_dir, filename), index=False)

In [4]:
calculate_toxicity_scores(filepaths[f"04_{dataset}_scored"])

100%|███████████████████████████████████████████████████████████████| 142/142 [00:14<00:00,  9.50it/s]
